In [1]:
# import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json

In [2]:
# Load data from JSON files
with open("memorizing_neurons_clean.json") as f:
    clean_neurons = json.load(f)
with open("memorizing_neurons_noisy.json") as g:
    noisy_neurons = json.load(g)

In [3]:
clean_neurons["conv1_1"][0]

[]

In [4]:
clean_inf_neurons = {}
for layer, datapoints in clean_neurons.items():
    if layer == "FC_softmax":
        continue
    neurons = {}
    for idx, datapoint in enumerate(datapoints):

        if len(datapoint) != 0:
            neurons.update({idx: datapoint})

        clean_inf_neurons.update({layer: neurons})

In [5]:
noisy_inf_neurons = {}
for layer, datapoints in noisy_neurons.items():
    if layer == "FC_softmax":
        continue
    neurons = {}
    for idx, datapoint in enumerate(datapoints):

        if len(datapoint) != 0:
            neurons.update({idx: datapoint})

        noisy_inf_neurons.update({layer: neurons})

In [6]:
# neuron overlap
noisy_inf_neurons

{'conv1_1': {121: [6, 9, 22, 29],
  462: [25],
  786: [6, 10, 12, 13, 17],
  800: [2],
  962: [13, 17, 22, 27, 29]},
 'conv1_2': {119: [16, 20, 27],
  121: [0, 3, 4, 8, 10, 13, 15, 19, 29],
  148: [20],
  191: [12, 16, 23],
  222: [16, 23],
  287: [16, 20, 21, 23, 27],
  303: [24],
  373: [20],
  374: [0, 8, 10, 12, 23, 29],
  393: [16],
  424: [3, 23, 27],
  462: [0, 3, 4, 7, 8, 15, 17, 24, 25, 27],
  486: [16],
  535: [16],
  604: [0, 24],
  669: [16],
  761: [11, 12, 16, 20, 21, 23, 27],
  786: [4, 5, 6, 9, 11, 20, 24, 25, 27],
  800: [0, 4, 15, 20, 23, 25, 27],
  859: [3, 9, 20, 27],
  916: [16, 20],
  962: [7, 8, 12, 13, 16, 20, 21, 23, 26, 27]},
 'conv2_1': {110: [38],
  121: [0, 1, 7, 27, 29, 35, 36, 42, 47, 48, 52, 53, 54, 56, 59, 61, 63],
  148: [39],
  222: [0],
  287: [0, 19, 28, 30, 39, 60],
  374: [0, 6, 28, 45, 51, 60],
  424: [14, 19, 28, 39, 54, 60],
  462: [1,
   4,
   7,
   13,
   14,
   18,
   19,
   22,
   25,
   35,
   39,
   43,
   44,
   45,
   48,
   54,
   55,


In [12]:
noisy_layer_neuron = {}
for layer, inf_neurons in noisy_inf_neurons.items():
    imp_neurons = list(
        set(item for sublist in inf_neurons.values() for item in sublist)
    )
    noisy_layer_neuron[layer] = imp_neurons

In [24]:
clean_layer_neuron = {}
for layer, inf_neurons in clean_inf_neurons.items():
    imp_neurons = list(
        set(item for sublist in inf_neurons.values() for item in sublist)
    )
    clean_layer_neuron[layer] = imp_neurons

In [25]:
for key, value in clean_layer_neuron.items():
    print(key, len(value))

conv1_1 17
conv1_2 24
conv2_1 49
conv2_2 58
FC_1 75
FC_2 70


In [23]:
for key, value in noisy_layer_neuron.items():
    print(key, len(value))

conv1_1 11
conv1_2 24
conv2_1 45
conv2_2 52
FC_1 88
FC_2 81


In [26]:
overlapping_neurons = {}

for key in noisy_layer_neuron.keys():
    if key in clean_layer_neuron:
        # Find the overlapping values for the current key
        overlap = set(noisy_layer_neuron[key]) & set(clean_layer_neuron[key])
        overlapping_neurons[key] = list(overlap)

In [28]:
for key, value in overlapping_neurons.items():
    print(key, len(value))

conv1_1 9
conv1_2 20
conv2_1 41
conv2_2 50
FC_1 63
FC_2 60


In [ ]:
# change clean to noisy if needed
original_pred = model(clean_xTensors)

In [ ]:
import json


def get_neurons(sample):
    # define the layers to be used, since dropout or flatten don't have weights
    conv_layers = [
        model.get_layer("conv1_1"),
        model.get_layer("conv1_2"),
        model.get_layer("conv2_1"),
        model.get_layer("conv2_2"),
    ]  #
    fc_layers = [
        model.get_layer("FC_1"),
        model.get_layer("FC_2"),
        model.get_layer("FC_softmax"),
    ]
    predictionlist = []

    for conv_layer in range(len(conv_layers)):

        for layer, neurons in noisy_layer_neuron.items():
            # get original weights of layer
            original_weights = layer.get_weights()
            # zero out each neuron until pred flips
            if (
                layer.name == "conv1_1"
                or layer.name == "conv1_2"
                or layer.name == "conv2_1"
                or layer.name == "conv2_2"
            ):
                for neuron_index in neurons:
                    original_weights[0][:, :, :, neuron_index] = 0.0
                    layer.set_weights(original_weights)
                    # get prediction on this model
                    pred_0 = model(sample)
                    predictionlist.append(pred_0)

            elif (
                layer.name == "FC_1"
                or layer.name == "FC_2"
                or layer.name == "FC_softmax"
            ):
                for neuron_index in neurons:
                    original_weights[0][:, neuron_index] = 0.0
                    layer.set_weights(original_weights)
                    # get prediction on this model
                    pred_0 = model(sample)
                    pred_0 = tf.argmax(pred_0, axis=1)
                    predictionlist.append(pred_0)

    return predictionlist